### SahabatAI-v1-8B-Instruct LLM run on Aristotle pipeline
### This is run in Runpod/remote Jupyter environment

In [ ]:
!apt update

In [ ]:
!apt install git-lfs

In [ ]:
!pip install transformers safetensors sentencepiece huggingface-hub accelerate bitsandbytes tqdm openai backoff retrying protobuf

In [ ]:
!git lfs install

In [ ]:
!git clone https://huggingface.co/Sahabat-AI/llama3-8b-cpt-sahabatai-v1-instruct LLM_MODELS/llama3-8b-cpt-sahabatai-v1-instruct

In [ ]:
import os

os.environ["TOKENIZERS_PARALLELISM"] = "false"
######### Also useful to reduce thread contention:
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"

snapshot_path = "/workspace/LLM_MODELS/llama3-8b-cpt-sahabatai-v1-instruct" ############## <--- Change this based on platform and models
#snapshot_path = "Sahabat-AI/llama3-8b-cpt-sahabatai-v1-instruct"

os.environ["LOCAL_MODEL_PATH"] = snapshot_path
os.environ["LLM_MODEL"] = snapshot_path

######## enable 4-bit for quant (and bitsandbytes is set up)
os.environ["LLM_LOAD_IN_4BIT"] = "1"  # or "0" to disable quantization
print("LOCAL_MODEL_PATH =", os.environ["LOCAL_MODEL_PATH"])
print("LLM_MODEL =", os.environ["LLM_MODEL"])

In [ ]:
#Test for Max Time generation stopping criteria
from llm_backends import HFBackend
hb = HFBackend(local_model_path=snapshot_path, quantize_4bit=True)
print("Calling short test (3s max_time)...")
res = hb.generate("Write a long list of words and sentences: ", max_new_tokens=1024, max_time=3.0)
print("Result length:", len(res))
print(res[:1000])

print("Calling short test (10s max_time)...")
res = hb.generate("Write a long list of words and sentences: ", max_new_tokens=1024, max_time=10.0)
print("Result length:", len(res))
print(res[:1000])

In [ ]:
#!python run_pipeline.py --dataset_name ProntoQA --sample_pct 0 --batch_size 1
#print("\nFinished translating dataset\n")

In [ ]:
os.environ["LLM_WORKER_MAX_TIME"] = "450"  # Change this every process (translate, decompose, search_resolve), different time limit is needed in seconds.

# With original prompts
!python translate_to_fol.py --data_path manual_data_translated --dataset_name ProntoQA --sample_pct 0 --prompts_folder manual_prompts_translated --prompts_file translation --split dev --save_path results_translated/prompts_original --model_name $LLM_MODEL --batch_num 1 --max_new_tokens 8000


In [ ]:
# With modified prompts
!python translate_to_fol.py --data_path manual_data_translated --dataset_name ProntoQA --sample_pct 100 --prompts_folder manual_prompts_translated --prompts_file translation_modified --split dev --save_path results_translated/prompts_modified --model_name $LLM_MODEL --batch_num 1 --max_new_tokens 8000

In [ ]:
# With refined prompts
!python translate_to_fol.py --data_path manual_data_translated --dataset_name ProntoQA --sample_pct 100 --prompts_folder manual_prompts_translated --prompts_file translation_refine --split dev --save_path results_translated/prompts_refine --model_name $LLM_MODEL --batch_num 1 --max_new_tokens 8000

In [ ]:
os.environ["LLM_WORKER_MAX_TIME"] = "900"

#!python decompose_to_cnf.py --data_path manual_data_translated --dataset_name ProntoQA --prompts_folder manual_prompts_translated --split dev --save_path results_translated --model_name /workspace/LLM_MODELS/llama3-8b-cpt-sahabatai-v1-instruct --batch_num 1 --max_new_tokens 8000

In [ ]:
#!python negate.py --dataset_name ProntoQA --save_path results_translated --model_name /workspace/LLM_MODELS/llama3-8b-cpt-sahabatai-v1-instruct

In [ ]:
#!python search_resolve.py --data_path manual_data_translate --dataset_name ProntoQA --prompts_folder manual_prompts_translated --split dev --save_path results_translated --model_name /workspace/LLM_MODELS/llama3-8b-cpt-sahabatai-v1-instruct --batch_num 1 --negation False --max_new_tokens 8000

In [ ]:
#!python search_resolve.py --data_path manual_data_translate --dataset_name ProntoQA --prompts_folder manual_prompts_translated --split dev --save_path results_translated --model_name /workspace/LLM_MODELS/llama3-8b-cpt-sahabatai-v1-instruct --batch_num 1 --negation True --max_new_tokens 8000

In [ ]:
#!python evaluate.py --dataset_name ProntoQA --save_path results_translated --model_name /workspace/LLM_MODELS/llama3-8b-cpt-sahabatai-v1-instruct